# Span情報がない平均報酬強化学習

参考：
* [Achieving Tractable Minimax Optimal Regret in Average Reward MDPs](https://arxiv.org/abs/2406.01234)

Average RewardでのRLは一般にSpanの情報や半径の情報が必要になります．今回はSpanが不要なMDPについて学んでみましょう．

表記：
* MDP：$M \in \mathcal{M}$
* ゲイン：$g^\pi(s):=\lim \frac{1}{T} \mathbf{E}_s^\pi\left[R_0+\ldots+R_{T-1}\right]$
* バイアス：$h^\pi:=\lim \sum_{t=0}^{T-1}\left(R_t-g\left(S_t\right)\right)$
* Poisson方程式：$h^\pi+g^\pi=r^\pi+P^\pi h^\pi$
* ベルマン方程式：$L u(s):=\max _{a \in \mathcal{A}(s)}\{r(s, a)+p(s, a) u\}$
  * 今回はWeakly communicatingの設定を考える．つまり，$Lh^* - h^* \in \boldmath{R}e$を満たすような$h^*$が存在する．
  * これは任意の方策に対して$r^\pi+P^\pi h^* \leq g^*+h^*$を満たす
* ベルマン誤差：$\Delta^*(s, a):=h^*(s)+g^*(s)-r(s, a)-p(s, a) h^* \geq 0$
* 直径：$D:=\sup _{s \neq s^{\prime}} \inf _\pi \mathbf{E}_s^\pi\left[\inf \left\{t \geq 1: S_t=s^{\prime}\right\}\right]$
* リグレット：
    * $\operatorname{Reg}(T):=T g^*-\sum_{t=0}^{T-1} R_t$
    * $\mathbf{E}[\operatorname{Reg}(T)]=\mathbf{E}\left[\sum_{t=0}^{T-1} \Delta^*\left(X_t\right)\right]+\mathbf{E}\left[h^*\left(S_0\right)-h^*\left(S_T\right)\right]$
* SpanがバウンドされたMDPの集合：$\mathcal{M}_c:=\left\{M \in \mathcal{M}: \exists h^* \in \operatorname{Fix}(L(M)), \operatorname{sp}\left(h^*\right) \leq c\right\}$
  * 下界：$\max _{M \in \mathcal{M}_c} \mathbf{E}^{M, \mathbf{A}}[\operatorname{Reg}(T)]=\Omega(\sqrt{c S A T})$

## PMEVI-DT アルゴリズム

基本的なアイデアはOFUと同じです．OFUは次の楽観的なゲインを計算します：
$$
g^*\left(\mathcal{M}_t\right):=\sup \left\{g^\pi\left(\mathcal{M}_t\right): \pi \in \Pi, \operatorname{sp}\left(g^\pi\left(\mathcal{M}_t\right)\right)=0\right\} \text { with } g^\pi\left(\mathcal{M}_t\right):=\sup \left\{g(\pi, \widetilde{M}): \widetilde{M} \in \mathcal{M}_t\right\}
$$

OFUの更新タイミングはいろいろありますが，今回はDoubling trickを使います．つまり，
$$
N_t\left(S_t, \pi_k\left(S_t\right)\right) \geq 1 \vee 2 N_{t_k}\left(X_t\right)
$$
のタイミングで更新を行います（$X_t = (S_t, A_t)$）．

### Extended Value Iteration (EVI)について

UCRL2など，OFUを実現するためには基本的にEVIを使います．
$(s, a)$-rectangularな不確実集合$\mathcal{M}_t \equiv \prod_{s, a}\left(\mathcal{R}_t(s, a) \times \mathcal{P}_t(s, a)\right)$を作り，次の楽観的な作用素でバイアス関数を更新します：

$$
v_{i+1}(s) \equiv \mathcal{L}_t v_i(s):=\max _{a \in \mathcal{A}(s)} \max _{\tilde{r}(s, a) \in \mathcal{R}_t(s, a)} \max _{\tilde{p}(s, a) \in \mathcal{P}_t(s, a)}\left(\tilde{r}(s, a)+\tilde{p}(s, a) \cdot v_i\right)
$$

そして，スパンが$\operatorname{sp}\left(v_{i+1}-v_i\right)<\epsilon$を満たすまで繰り返すのがEVIです．このとき，$\mathcal{L}_t v_i$を与える方策は
$g^\pi\left(\mathcal{M}_t\right) \geq g^*(\mathcal{M})-\epsilon$を満たすことが知られています．


### Projected Mitigated EVI

基本的に，OFUは$\mathcal{M}_t$の良さによって実現されます．
よって，多くの先行研究は$\mathcal{M}_t$を改善できるように様々な工夫をこらしてました．

今回の論文は，あまり$\mathcal{M}_t$を改善することに固執していません．
* いい感じの挙動をする信頼区間を使い，
* バイアスの推定をして，
* EVIを改善する

ことで，Minimax最適なアルゴリズムを達成するのが今回の論文です．

これを説明するために，何らかの方法で，$h^*$を推定するためのバイアスの信頼区間$\mathcal{H}_t$が与えられているとします．$M\in \mathcal{M}_t$かつ$h^* \in \mathcal{H}_t$が満たされているならば，
* ゲインを最大化して，かつ$h(\pi, \tilde{M}) \in \mathcal{H}_t$を満たすような方策とMDPのペア$(\pi, \tilde{M})$を見つければ，OFUできそうな気がします．

そこで，「Projection」と「Mitigation」の２つのテクニックを使います．

1. Projection: もし$h^* \in \mathcal{H}_t$ならば，OFUで探す最適方策はバイアスが$\mathcal{H}_t$の中に入るものに限定して構いません．そこで，$\Gamma_t: \mathbf{R}^{\mathcal{S}} \rightarrow \mathcal{H}_t$を使ってバイアスを射影します．
2. Mitigation: 一旦ボーナスベースのアルゴリズムについて考えてみましょう．ボーナスベースのアルゴリズムは，
$$
\tilde{p}(s, a) u_i \leq \hat{p}_t(s, a) u_i+\underbrace{\left(p(s, a)-\hat{p}_t(s, a)\right) u_i}_{\leq ボーナス関数}
$$
によって，$推定した遷移\cdot 価値+ボーナス$を使ってOFUを実現します．
今回のアルゴリズムはこれを利用します．
もし$h^* \in \mathcal{H}_t$ならば，$\beta_t(s, a):=\max _{u \in \mathcal{H}_t} \beta_t(s, a, u)$とすれば，$h^*$がわからなくても，$\left(\hat{p}_t(s, a)-p(s, a)\right) h^* \leq \beta_t(s, a)$が成立します．
これを使って，次のEVIを後で利用します．
$$
\mathcal{L}_t^\beta u(s):=\max _{a \in \mathcal{A}(s)} \sup _{\tilde{r}(s, a) \in \mathcal{R}_t(s, a)} \sup _{\tilde{p}(s, a) \in \mathcal{P}_t(s, a)}\left\{\tilde{r}(s, a)+\min \left\{\tilde{p}(s, a) u_i, \hat{p}_t(s, a) u_i+\beta_t(s, a)\right\}\right\}
$$

---

上のProjectionとMitigationを踏まえて，今回のアルゴリズムでは次の「MitigateしてProjection」を繰り返します：

$$\mathfrak{L}_t:=\Gamma_t \circ \mathcal{L}_t^\beta$$

これはCompositionなので，うまく動くかは自明ではありません．しかし，次の定理によって挙動が保証されます：

